In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Configuration
HEIGHT = 180
WIDTH = 320
MAX_STEER_DEGREES = 40
BATCH_SIZE = 32
DATA_DIR = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/data/navi/seg'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Preprocessing
class CustomImageDataset(Dataset):
    def __init__(self, image_files):
        self.image_files = image_files
        self.transform = transforms.Compose([
            transforms.Resize((HEIGHT, WIDTH)),
            transforms.ToTensor(),  # Converts to [0, 1] range
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = float(os.path.basename(image_path).split('.png')[0].split('_')[2])
        if label > MAX_STEER_DEGREES:
            label = MAX_STEER_DEGREES
        elif label < -MAX_STEER_DEGREES:
            label = -MAX_STEER_DEGREES
        label = label / MAX_STEER_DEGREES  # Normalize label

        input_2 = int(os.path.basename(image_path).split('.png')[0].split('_')[1])

        # Load and preprocess the image
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)

        return image, torch.tensor(input_2, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# Model Definition
class AutoNaviModel(nn.Module):
    def __init__(self):
        super(AutoNaviModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * (HEIGHT // 8) * (WIDTH // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 4),
            nn.ReLU(),
            nn.Flatten()
        )
        self.output_layer = nn.Linear(5, 1)  # 4 from image + 1 from input_2

    def forward(self, image, navigational_direction):
        image_features = self.conv_layers(image)
        combined = torch.cat((image_features, navigational_direction.unsqueeze(1)), dim=1)
        output = self.output_layer(combined)
        return output


In [11]:
# Load Data
image_files = [os.path.join(DATA_DIR, file) for file in os.listdir(DATA_DIR) if file.endswith('.png')]
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files, val_files = image_files[:split_index], image_files[split_index:]

train_dataset = CustomImageDataset(train_files)
val_dataset = CustomImageDataset(val_files)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model, Loss, Optimizer
model = AutoNaviModel().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Continuosly
save_path = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/auto_navi_model_2.pth'

model = torch.load(save_path)

EPOCHS = 10
# Training Loop
# for epoch in range(EPOCHS):
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images, navigational_direction, labels in train_loader:
        images, navigational_direction, labels = images.to(DEVICE), navigational_direction.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images, navigational_direction)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        print(loss)
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for images, navigational_direction, labels in val_loader:
            images, navigational_direction, labels = images.to(DEVICE), navigational_direction.to(DEVICE), labels.to(DEVICE)
            outputs = model(images, navigational_direction)
            loss = criterion(outputs.squeeze(), labels)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Validation Loss: {avg_val_loss:.4f}")


# Save model and optimizer states
save_path = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/auto_navi_model_3.pth'
torch.save(model, save_path)


<ipython-input-11-575c4f08c16f>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(save_path)


tensor(0.0277, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0059, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0164, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0066, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0060, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0090, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0093, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0059, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0127, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0288, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0135, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0154, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0124, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0034, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0114, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0043, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0063, device='cuda:0', grad_fn=

In [9]:
# Save model and optimizer states
save_path = '/content/drive/MyDrive/Thesis/Self-Driving-Car/Auto-Navi/models/auto_navi_model_2.pth'
torch.save(model, save_path)